# Example of Running `tweakwcs` to Align JWST images

***
## About this Notebook
**Author:** Mihai Cara, STScI
<br>**Updated On:** 09/11/2018

***
## Imports

In [ ]:
from copy import deepcopy
from os import path

import numpy as np
from astropy.table import Column, Table
from astropy.modeling.models import RotateNative2Celestial

from jwst.datamodels import ImageModel
from tweakwcs import tweak_image_wcs, tweak_wcs, TPMatch, JWSTgWCS
from photutils import detect_threshold, DAOStarFinder

***
## Load Data:

In [ ]:
data_dir = '/Users/mcara/repo/tweakwcs/devdata/data' # replace with actual path to data

# Load JWST image models:
im1 = ImageModel(path.join(data_dir, 'jw10002001001_01101_00001_nrcb1_cal.fits'))
im2 = ImageModel(path.join(data_dir, 'jw10002001001_01102_00001_nrcb1_cal.fits'))

# save a copy of im1's wcs for later use in EXAMPLE 2:
im1_gwcs = deepcopy(im1.meta.wcs)

***
## EXAMPLE 1: Typical Workflow to Align Two or More Images

### 1. Create Source Catalogs

In [ ]:
for catno, im in enumerate([im1, im2]):
    threshold = detect_threshold(im.data, snr=5.0)[0, 0]
    daofind = DAOStarFinder(fwhm=2.5, threshold=threshold)
    cat = daofind(im.data)
    cat.rename_column('xcentroid', 'x')
    cat.rename_column('ycentroid', 'y')
    cat.meta['name'] = 'im{:d} sources'.format(catno)
    im.meta['catalog'] = cat
    # Assign group_id when aligning images that belong to the same SCA:
    # im.meta['group_id'] = 1

### 2. Align Images Using Image Source Catalogs

In [ ]:
tweak_image_wcs([im1, im2])

### 3. Inspect Corrected WCS. Save Aligned Image.

In [ ]:
# inspect WCS Tangent-Plane Corrections
for f, p in im2.meta.wcs.pipeline:
    if f == 'v2v3':
        print(p.__repr__())
        break

# save the file:
im2.write('im2_aligned.fits')

***
## A Customizable Workflow to Align Two Simulated Catalogs

In this example we assume that an image WCS contains errors by generating an image catalog artificially displaced with regard to a perfect reference catalog (obtained using this uncorrected/initial image WCS). We then use `tweakwcs` to find corrections to the image WCS to make the catalogs align.

### 1. Generate Reference Catalog

In [ ]:
ny, nx = im1.data.shape

# generate random sources in im1 detector:
rcx = Column(data=np.random.randint(0, nx, 30), name='x', dtype=np.float)
rcy = Column(data=np.random.randint(0, ny, 30), name='y', dtype=np.float)

# convert image coordinates to sky coords:
ra, dec = im1_gwcs(rcx, rcy)
rra = Column(data=ra, name='RA', dtype=np.float)
rdec = Column(data=dec, name='DEC', dtype=np.float)

# create the catalog:
refcat = Table([rra, rdec])
refcat.meta['name'] = 'REFCAT'

### 2. Create Image Catalog

We take the reference catalog and apply small Euler rotations to sky positions. This "displaced" catalog now will serve as an image catalog that we will try to align back to the reference catalog and find the correction that needs to be applied to the "image's WCS" to make image catalog align to the reference catalog. Essentially, here we simulate an image source catalog as if these sources were detected in image `im2` having the WCS from `im1.meta.wcs`.

In [ ]:
# Apply small Euler rotations to the reference source coordinates:
m = RotateNative2Celestial(5.0e-05, 90, 180)
ra_displ, dec_displ = m(ra, dec)

# convert sky coordinates to image coordinates using initial/reference image WCS:
x_displ, y_displ = im1_gwcs.invert(ra_displ, dec_displ)

# remove sources outside the detector:
mask = (x_displ > 0) & (x_displ < nx) & (y_displ > 0) & (y_displ < ny)
x_displ = x_displ[mask]
y_displ = y_displ[mask]

# Create image catalog also randomizing source order:
imcat = Table([x_displ, y_displ], names=['x', 'y'])
imcat.meta['name'] = 'IMCAT'

### 3. Create a telescope/instrument-specific "corrector" WCS object

In [ ]:
# create JWST-specific TPWCS object using the WCS that needs to be corrected:
wcsinfo = im1.meta.wcsinfo._instance
imwcs = JWSTgWCS(wcs=im1_gwcs, wcsinfo=wcsinfo)

### 4. Find Indices of Matched Sources

In [ ]:
# Match sources in the catalogs:
match = TPMatch(searchrad=5, separation=0.1, tolerance=5, use2dhist=False)
ridx, iidx = match(refcat, imcat, imwcs)

### 5. Align Images:

In [ ]:
tweaked_tpwcs = tweak_wcs(refcat[ridx], imcat[iidx], imwcs)
corrected_wcs = tweaked_tpwcs.wcs

### 6. Inspect Corrected WCS. Save Aligned Image.

In [ ]:
# inspect WCS Tangent-Plane Corrections
for f, p in corrected_wcs.pipeline:
    if f == 'v2v3':
        print(p.__repr__())
        break

# assign tweaked WCS as model's WCS:
im2.meta.wcs = corrected_wcs

# save the file:
im2.write('im2_aligned.fits')